# Exploratory data analysis of the wine quality dataset

## Summary of the Dataset

The dataset for this project is sourced from [UCI Machine Learning Repository](https://archive-beta.ics.uci.edu/) and specifically [Wine Quality](https://archive-beta.ics.uci.edu/ml/datasets/wine+quality). It was created by Cortez, Paulo, Cerdeira, A., Almeida, F., Matos, T. & Reis, J.. (2009). Wine Quality. UCI Machine Learning Repository.

The link includes two datasets for red and white wine. The two datasets are related to red and white variants of the Portuguese "Vinho Verde" wine. For more details, consult: http://www.vinhoverde.pt/en/ or the reference [Cortez et al., 2009]. Each row in the datasets represent a wine, and there are 6497 observations in total. There are 12 features in the combined dataset, and we will use these features to classify the quality of the wine. There are 7 different categories of quality in the dataset. The classes are ordered but not balanced (e.g. there are many more normal wines than excellent or poor ones). Also, there are some outliers in the dataset, but there is no missing value. 

In [1]:
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split
alt.renderers.enable('mimetype')

In [2]:
white = pd.read_csv("../data/raw/winequality-white.csv", delimiter=";")
white['type'] = 'white'
white['quality'] = white['quality'].astype('string')

In [3]:
red = pd.read_csv("../data/raw/winequality-red.csv", delimiter=";")
red['type'] = 'red'
red['quality'] = red['quality'].astype('string')

In [4]:
data = pd.concat((white, red))
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,white
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,white
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,red
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,red
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,red
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,red


Table1. Combined Dataset

# Partition the data set into training and test sets

Before going further, we split the data into training and testing portions. The training portion includes 70% of the observations and the testing portion includes the remaining of the data. There is a class imbalance in the dataset and below is the list of the counts of observations for each class. There is a class imbalance in the dataset, and we will address that during the classification. The quality 5 and 6 which are the normal kinds of wine have the largest counts. We will consider some methods like balancing the classes or using performance metrics (confusion matrix, recall, precision and f1-score) for addressing class imbalance and pick one of them.

In [5]:
train_df, test_df = train_test_split(data, test_size=0.3, random_state=123)

In [6]:
counts = pd.DataFrame(train_df['quality'].value_counts()).reset_index()
counts.columns = ['Quality score', "Counts"]
counts.sort_values(by = "Quality score").set_index(counts.columns[0])

,Counts
Quality score,
3,22
4,157
5,1514
6,1969
7,756
8,126
9,3


Table2. Count of Observations for each Class in the Training Dataset

## Exploratory analysis on the training data set

To find the relevant features we plotted the distribution of each feature in the training set and colored them by the quality of the wine. We observed that among all the features, alcohol, volatile acidity, residual sugar, total sulfur dioxide and density are the most influential features. Moreover, we observed that there is some difference in terms of the mean and standard deviation of each feature for each class.

In [7]:
summary = train_df.groupby('quality').agg(['mean', 'std']).round(2)
summary

fixed acidity       volatile acidity       citric acid        \
                 mean   std             mean   std        mean   std   
quality                                                                
3                8.00  1.70             0.56  0.34        0.28  0.19   
4                7.31  1.27             0.46  0.23        0.27  0.18   
5                7.33  1.26             0.39  0.18        0.31  0.16   
6                7.19  1.32             0.32  0.15        0.32  0.14   
7                7.18  1.41             0.29  0.12        0.34  0.11   
8                6.88  1.15             0.30  0.12        0.33  0.10   
9                7.13  0.25             0.29  0.06        0.40  0.08   

        residual sugar       chlorides        ... total sulfur dioxide  \
                  mean   std      mean   std  ...                 mean   
quality                                       ...                        
3                 4.46  4.60      0.08  0.07  ...               123.14   
4                 4.11  3.88      0.06  0.05  ...               104.77   
5                 5.80  4.96      0.06  0.04  ...               121.02   
6                 5.56  4.91      0.05  0.03  ...               115.45   
7                 4.75  4.04      0.05  0.02  ...               109.66   
8                 5.19  3.83      0.04  0.02  ...               118.46   
9                 2.80  1.22      0.03  0.01  ...               123.67   

                density         pH       sulphates       alcohol        
            std    mean  std  mean   std      mean   std    mean   std  
quality                                                                 
3        124.83    1.00  0.0  3.25  0.22      0.51  0.11   10.11  1.04  
4         62.64    0.99  0.0  3.23  0.19      0.52  0.18   10.16  1.00  
5         61.67    1.00  0.0  3.21  0.16      0.53  0.14    9.83  0.81  
6         55.27    0.99  0.0  3.22  0.16      0.53  0.15   10.59  1.13  
7         47.99    0.99  0.0  3.22  0.16      0.55  0.16   11.39  1.21  
8         40.06    0.99  0.0  3.22  0.16      0.51  0.15   11.72  1.28  
9         13.61    0.99  0.0  3.31  0.05      0.42  0.06   12.70  0.20  

[7 rows x 22 columns]

Table3. Summary of Statistics for each Feature

This figure shows the distribution of the wine quality:

In [8]:
alt.Chart(train_df, title='Class Imbalance of Wine Quality').mark_bar().encode(
    alt.X('quality', title='Quality'), 
    alt.Y('count()', title='Counts'),
    alt.Color('quality', title='Quality')).properties(width=350, height=350)

alt.Chart(...)

Figure1. Class Imbalance of Wine Quality

This figure shows the distribution of the white and red wine in the dataset:

In [9]:
alt.Chart(train_df, title='Red and White Wine Quantities').mark_bar(opacity=0.5).encode(
    alt.X('quality', title='Quality'),
    alt.Y('count()', title='Counts', stack=False),
    alt.Color('type', title='Type')).properties(width=350, height=350)

alt.Chart(...)

Figure2. Red and White Wine Quantities

These charts represent the distribution of each feature colored by the wine quality:

In [10]:
alt.Chart(train_df).mark_line(interpolate='step').encode(
    alt.X(alt.repeat(), type='quantitative', bin=alt.Bin(maxbins=40)),
    alt.Y('count()', title='Counts'), 
    alt.Color('quality', title='Quality')).properties(width=300, height=450).repeat(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'], columns=3)

alt.RepeatChart(...)

Figure3. Distribution of each Feature Colored by Wine Quality

# References

Cortez, Paulo, Cerdeira, A., Almeida, F., Matos, T. & Reis, J.. (2009). Wine Quality. UCI Machine Learning Repository. [Wine Quality](https://archive-beta.ics.uci.edu/ml/datasets/wine+quality)